This notebook produces an interactive plot of the LSST MAF skymap results for our bayes factor metric alongside a slope function, to demonstrate the connection between the two.  

In [ ]:
%matplotlib notebook 
#ABOVE: enables matplotlib interactivity in the notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.widgets import Slider

This cell loads the datafiles that represent the slope function in each band and splits them into two arrays for plotting purposes.

In [ ]:
gband = np.loadtxt('gbandslopes.dat', usecols=range(2,1))
uband = np.loadtxt('ubandslopes.dat', usecols=range(2,1))
gbandslopes, gzshift = np.split(gband, 2, axis = 1)
ubandslopes, uzshift = np.split(uband, 2, axis = 1)

**IMPORTANT:  Change this value to swap between the g and u filters.  Only accepts 'u' and 'g' 

In [ ]:
band = 'g'
if band != 'u' and band != 'g':
    print 'ERROR: Band must be either u or g'


This cell defines the update function for the slider, which plots the appropriate skymaps based on the slider's value.

In [ ]:
oldval = 0.4 #this is used later to reduce the computation necessary in the update function
def update(val):
    global oldval
    if szshift.val >= oldval + 0.05 or szshift.val <= oldval - 0.05:
        #clear old plots
        ax.cla()
        imgplot.cla()
        imgplot2.cla()
        #reset labels
        ax.set_xlabel('Redshift')
        ax.set_ylabel('Slope of R vs. Tan(Z)')
        #plot band-specific features
        if band == 'g':
            zpoint = gbandslopes[int(round((szshift.val/.05)-2))]
            ax.plot(gzshift, gbandslopes)
        if band == 'u':
            zpoint = ubandslopes[int(round((szshift.val/.05)-2))]
            ax.plot(uzshift, ubandslopes)
        ax.plot(szshift.val, zpoint, 'ko') #current redshift point
        ax.plot(xaxis, y) #plot y=0 line
        #reset filepaths
        imgpath = 'MAFthumbs/z' + str(round(szshift.val, 1)) + '/' + band + 'band/'
        imgname = 'thumb.opsim_astrometry_' + band + '_HEAL_kraken_1042_SkyMap.png'
        imgname2 = 'thumb.opsim_astrometry_' + band + '_and_night_lt_366_HEAL_kraken_1042_SkyMap.png'
        img = mpimg.imread(imgpath + imgname)
        img2 = mpimg.imread(imgpath + imgname2)
        #plot skymap thumbnails for nearest applicable value
        imgplot.imshow(img)
        imgplot2.imshow(img2)
        oldval = szshift.val
    fig.canvas.draw_idle()

This cell sets up the initial condition of the plot and starts the interactive slider widget.  Note that there are only skymaps for each 0.1 interval of redshift, so not every value on the slider has a skymap associated.  This results in apparent inaccuracies, especially in the areas halfway between two skymaps.  

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle(band + 'band')
ax = fig.add_subplot() #subplot for slope function
ax = plt.subplot2grid((8,6), (0,0), colspan = 6, rowspan = 2)
ax.set_xlabel('Redshift')
ax.set_ylabel('Slope of R vs. Tan(Z)')
ax.xaxis.set_visible(False)
#plot setup for y=0 line
numb = 100
xaxis = np.linspace(0.1, 4.0, numb)
y = 0*xaxis
#setup for image handling; imgplot is 10 year, imgplot2 is 1 year
imgplot = plt.subplot2grid((8,6), (2, 3), colspan = 3, rowspan = 5)
imgplot.xaxis.set_visible(False)
imgplot.yaxis.set_visible(False)
imgplot2 = plt.subplot2grid((8,6), (2, 0), colspan = 3, rowspan = 5)
imgplot2.xaxis.set_visible(False)
imgplot2.yaxis.set_visible(False)
f0 = 0.4 #initial value of the slider/redshift
#filepaths for skymap thumbnails
imgpath = 'MAFthumbs/z' + str(f0) + '/' + band + 'band/'
imgname = 'thumb.opsim_astrometry_' + band + '_HEAL_kraken_1042_SkyMap.png'
imgname2 = 'thumb.opsim_astrometry_' + band + '_and_night_lt_366_HEAL_kraken_1042_SkyMap.png'
img = mpimg.imread(imgpath + imgname)
img2 = mpimg.imread(imgpath + imgname2)
#initial plotting
if band == 'g':
    zpoint = gbandslopes[int(round((f0/.05)-2))]
    ax.plot(gzshift, gbandslopes)
if band == 'u':
    zpoint = ubandslopes[int(round((f0/.05)-2))]
    ax.plot(uzshift, ubandslopes)
ax.plot(f0, zpoint, 'ko') #point at initial redshift
ax.plot(xaxis, y)
imgplot.imshow(img)
imgplot2.imshow(img2)
#setup for slider
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.16, 0.68, 0.693, 0.02], axis_bgcolor=axcolor)
sl_xticks = np.arange(0.1,3.9, 0.2)
szshift = Slider(axfreq, '', 0.1, 3.9, valinit=f0)
axfreq.xaxis.set_visible(True)
axfreq.set_xticks(sl_xticks)
szshift.on_changed(update)
plt.show()

The slider, despite its name, isn't very good at sliding.  It's much better to just click on the slider where you want to see, rather than sliding to it.  